# Diffusion Transformer

## Libraries

In [1]:
import os
import gc
import math
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torchvision import transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset
from accelerate import Accelerator
from tqdm.auto import tqdm
from einops import rearrange

import matplotlib.pyplot as plt
from ema_pytorch import EMA
from PIL import Image
from torchvision.transforms import ToPILImage

## Data preprocess

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, npy_path: str, image_size: int):
        # 1) load the preprocessed numpy array
        #    expected shape: (N, H, W, C), dtype=uint8 or float32
        self.sprites = np.load(npy_path)
        self.image_size = image_size

        # 2) expose .sprites_shape for legacy code
        #    (N, H, W, C)
        self.sprites_shape = self.sprites.shape

        # 3) augmentation / normalization pipeline
        self.transform = transforms.Compose([
            ToPILImage(),  # turn the H×W×C array into a PIL Image
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomResizedCrop(image_size, scale=(0.9, 1.0)),
            transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
            transforms.ToTensor(),                   # ➔ [0,1]
            transforms.Normalize((0.5, 0.5, 0.5),     # ➔ [–1,1]
                                 (0.5, 0.5, 0.5)),
        ])

    def __len__(self):
        return self.sprites_shape[0]

    def __getitem__(self, idx):
        # grab a single H×W×C numpy array
        img_np = self.sprites[idx]
        # apply your transforms (including ToPILImage internally)
        img = self.transform(img_np)
        return img

# Define a wrapper if using Hugging Face or custom VAE
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

# Example: load VAE from diffusers
from diffusers import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").eval().cuda()  # or your custom VAE
vae.requires_grad_(False)

class CustomLatentDataset(Dataset):
    def __init__(self, npy_path, image_size, vae):
        self.vae = vae
        self.images = np.load(npy_path).to("cuda")  # shape: [N, H, W, 3] or [N, 3, H, W]
        self.image_size = image_size

        # If images are in [N, H, W, 3], convert later
        self.is_channel_last = (self.images.shape[-1] == 3)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),                     # [0, 1]
            transforms.Normalize([0.5], [0.5])          # to [-1, 1]
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]

        # Convert channel-last to channel-first
        if self.is_channel_last:
            img = np.transpose(img, (2, 0, 1))  # HWC -> CHW

        img = torch.from_numpy(img).float()  # [3, H, W]
        img = self.transform(img)            # [3, image_size, image_size]

        # Add batch dim and push to CUDA for VAE
        img = img.unsqueeze(0).cuda()

        with torch.no_grad():
            latent = self.vae.encode(img).latent_dist.sample() * 0.18215

        return latent.squeeze(0)  # [4, H/8, W/8]


2025-05-21 01:56:15.216639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747763775.233822     842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747763775.238922     842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 01:56:15.256304: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

## Noise Generation

In [ ]:
def linear_beta_schedule(timesteps):
    """
    linear schedule, proposed in original paper
    """
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype = torch.float64)

# e.g. from Nichol & Dhariwal’s “improved DDPM”:
def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return betas.clamp(max=0.999)

def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

## network modules

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=8, emb_dim=512, img_size=64):
        super().__init__()
        self.patch_size = patch_size
        self.emb_dim = emb_dim
        self.img_size = img_size
        self.proj = nn.Conv2d(in_channels, emb_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embedding = nn.Parameter(torch.randn((img_size // patch_size)**2, emb_dim))

    def forward(self, x):
        x = self.proj(x)  # [B, emb_dim, H/patch, W/patch]
        x = rearrange(x, 'b c h w -> b (h w) c')
        return x + self.pos_embedding

class ConvPatchEmbedding(nn.Module):
    def __init__(self, in_channels, emb_dim, img_size, patch_size=2, stride=1):
        super().__init__()
        self.patch_size = patch_size
        self.stride     = stride

        # 1) small conv-stem: in → emb_dim, keeps H×W
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, emb_dim//2, kernel_size=3, padding=1, stride=1),
            nn.GELU(),
            nn.Conv2d(emb_dim//2, emb_dim,   kernel_size=3, padding=1, stride=1),
            nn.GELU(),
        )

        # 2) Unfold to get sliding 2×2 patches
        self.unfold = nn.Unfold(kernel_size=patch_size, stride=stride)

        # compute number of patches = ((H - P)//S + 1)**2
        num_patches_per_dim = (img_size - patch_size) // stride + 1
        num_patches = num_patches_per_dim**2

        # 3) project each flattened patch → emb_dim
        self.proj = nn.Linear(in_channels * patch_size * patch_size, emb_dim)

        # 4) learned positional embeddings
        self.pos_emb = nn.Parameter(torch.randn(1, num_patches, emb_dim))

    def forward(self, x):
        """
        x: [B, 3, 16, 16]
        → stem: [B, emb_dim, 16, 16]
        → unfold: [B, emb_dim*P*P, N_p]
        → proj:    [B, N_p, emb_dim]
        → +pos_emb: [B, N_p, emb_dim]
        """
        B = x.shape[0]
        x = self.stem(x)                          # [B, D, H, W]
        patches = self.unfold(x)                  # [B, D*P*P, N]
        patches = patches.permute(0, 2, 1)        # [B, N, D*P*P]
        tokens = self.proj(patches)               # [B, N, D]
        return tokens + self.pos_emb              # [B, N, D]

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = torch.exp(
            torch.arange(half_dim, device=device) * -(math.log(10000) / (half_dim - 1))
        )
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb  # [B, dim]

class TransformerBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        emb_dim: int,
        num_heads: int,
        dim_feedforward: int | None = None,
        dropout: float = 0.0
    ):
        """
        A single ViT‐style block that:
          1) optionally projects in_channels → emb_dim via 1×1 conv
          2) adds a (B,emb_dim) time embedding
          3) runs multihead self‐attention + residual + LayerNorm
          4) runs MLP (feed‐forward) + residual + LayerNorm
          5) optionally projects emb_dim → in_channels via 1×1 conv
        """
        super().__init__()
        # 1×1 conv to match channels → emb_dim (if needed)
        self.input_proj = (
            nn.Conv2d(in_channels, emb_dim, kernel_size=1)
            if in_channels != emb_dim else nn.Identity()
        )
        # Multi‐head self‐attention
        self.attn = nn.MultiheadAttention(
            embed_dim=emb_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm1 = nn.LayerNorm(emb_dim)
        # Feed-forward
        ff_dim = dim_feedforward or (emb_dim * 4)
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, emb_dim),
        )
        self.norm2 = nn.LayerNorm(emb_dim)
        # Project back to in_channels if needed
        self.output_proj = (
            nn.Conv2d(emb_dim, in_channels, kernel_size=1)
            if in_channels != emb_dim else nn.Identity()
        )

    def forward(self, x: torch.Tensor, t_emb: torch.Tensor) -> torch.Tensor:
        """
        x:     [B, in_channels, H, W]
        t_emb: [B, emb_dim]  (output of your time‐MLP)
        returns [B, in_channels, H, W]
        """
        B, C, H, W = x.shape

        # 1) project to emb_dim
        x0 = self.input_proj(x)          # [B, emb_dim, H, W]

        # 2) add time embedding (broadcast to spatial dims)
        t = t_emb[:, :, None, None]      # [B, emb_dim, 1, 1]
        x0 = x0 + t                      # [B, emb_dim, H, W]

        # 3) flatten to (B, N, emb_dim) for attention
        x_flat = rearrange(x0, 'b c h w -> b (h w) c')  # N = H*W

        # 4) self‐attention + residual + LN
        attn_out, _ = self.attn(x_flat, x_flat, x_flat, need_weights=False)
        x1 = self.norm1(x_flat + attn_out)

        # 5) feed-forward + residual + LN
        ff_out = self.ff(x1)
        x2 = self.norm2(x1 + ff_out)

        # 6) reshape back to [B, emb_dim, H, W]
        x3 = rearrange(x2, 'b (h w) c -> b c h w', h=H, w=W)

        # 7) project back to in_channels if needed
        out = self.output_proj(x3)       # [B, in_channels, H, W]
        return out


class TransformerBackbone(nn.Module):
    def __init__(self, in_channels=3, img_size=64, patch_size=2, stride=1, emb_dim=512, depth=6, num_heads=8, time_emb_dim=128):
        super().__init__()
        self.channels = in_channels
        self.embed = ConvPatchEmbedding(
            in_channels, emb_dim, img_size,
            patch_size=patch_size, stride=stride
        )

        self.time_embedding = SinusoidalPosEmb(time_emb_dim)
        self.time_mlp = nn.Sequential(
            nn.Linear(time_emb_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim)
        )

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=num_heads, dim_feedforward=1024, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.output_proj = nn.Sequential(
            nn.Linear(emb_dim, patch_size * patch_size * in_channels),
        )
        self.patch_size = patch_size
        self.img_size = img_size
        

    def forward(self, x, t):
        B = x.size(0)
        patches = self.embed(x)  # [B, N, D]

        t_emb = self.time_embedding(t) # [B, time_emb_dim]
        t_emb = self.time_mlp(t_emb) # [B, D]
        t_emb = t_emb.unsqueeze(1) # [B, 1, D]

        patches = patches + t_emb # Broadcast time embedding to all patches
        
        encoded = self.encoder(patches)  # [B, N, D]
        decoded = self.output_proj(encoded)  # [B, N, patch*patch*channels]
        decoded = decoded.view(B, -1, self.channels, self.patch_size, self.patch_size)  # reshape into patches
        h = w = self.img_size // self.patch_size
        decoded = rearrange(decoded, 'b (h w) c ph pw -> b c (h ph) (w pw)', h=h, w=w)
        return decoded
    
class ConvSkipViT(nn.Module):
    def __init__(
        self,
        in_channels=3,
        img_size=64,
        patch_size=2,
        stride=1,
        emb_dim=512,
        depth=6,
        num_heads=8,
        time_emb_dim=128,
    ):
        super().__init__()
        # 1) conv to lift 3→emb_dim, preserving H×W
        self.conv_in = nn.Conv2d(in_channels, emb_dim, kernel_size=3, padding=1)
        
        # 2) the Transformer expects emb_dim channels
        self.vit = TransformerBackbone(
            in_channels=emb_dim,
            img_size=img_size,
            patch_size=patch_size,
            stride=stride,
            emb_dim=emb_dim,
            depth=depth,
            num_heads=num_heads,
            time_emb_dim=time_emb_dim,
        )
        
        # 3) conv to project emb_dim→3 and add back the original RGB
        self.conv_out = nn.Conv2d(emb_dim, in_channels, kernel_size=3, padding=1)
        
        # for diffusion wrapper
        self.channels = in_channels
        self.image_size = img_size

    def forward(self, x, t):
        """
        x: [B, 3, H, W], t: [B] timesteps
        """
        # conv skip-in
        x_emb = self.conv_in(x)                     # [B, emb_dim, H, W]
        
        # pass through your ViT
        y = self.vit(x_emb, t)                      # [B, emb_dim, H, W]
        
        # conv skip-out + residual
        out = self.conv_out(y) + x                  # [B, 3, H, W]
        return out
    
class DoubleConv(nn.Module):
    """Two successive 3×3 convs each followed by GELU."""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.GELU(),
        )
    def forward(self, x):
        return self.net(x)

class UNetTransformer(nn.Module):
    def __init__(
        self,
        *,
        in_channels=3,
        base_channels=64,
        emb_dim=256,
        num_heads=8,
        time_emb_dim=128
    ):
        super().__init__()
        self.channels = in_channels

        # Time‐MLP
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
        )

        # ---- Encoder ----
        # Stage 1: 3 → 64
        self.enc1    = DoubleConv(in_channels,        base_channels)
        self.trans1  = TransformerBlock(base_channels, emb_dim, num_heads)

        # Stage 2: 64 → 128
        self.enc2    = DoubleConv(base_channels,      base_channels*2)
        self.down2   = nn.Conv2d(base_channels*2,     base_channels*2, 4, stride=2, padding=1)
        self.trans2  = TransformerBlock(base_channels*2, emb_dim, num_heads)

        # Stage 3: 128 → 256
        self.enc3    = DoubleConv(base_channels*2,    base_channels*4)
        self.down3   = nn.Conv2d(base_channels*4,     base_channels*4, 4, stride=2, padding=1)
        self.trans3  = TransformerBlock(base_channels*4, emb_dim, num_heads)

        # Stage 4: 256 → 512
        self.enc4    = DoubleConv(base_channels*4,    base_channels*8)
        self.down4   = nn.Conv2d(base_channels*8,     base_channels*8, 4, stride=2, padding=1)
        self.trans4  = TransformerBlock(base_channels*8, emb_dim, num_heads)

        # ---- Decoder ----
        # Upsample + concat + DoubleConv + Transformer per stage
        # Stage 4→3: 512 → 256, then cat with enc4(256)→512→256
        self.up4       = nn.ConvTranspose2d(base_channels*8, base_channels*4, 4, stride=2, padding=1)
        self.dec4      = DoubleConv(base_channels*4 + base_channels*8, base_channels*4)
        self.trans_up3 = TransformerBlock(base_channels*4, emb_dim, num_heads)

        # Stage 3→2: 256 → 128, then cat with enc3(128)→256→128
        self.up3       = nn.ConvTranspose2d(base_channels*4, base_channels*2, 4, stride=2, padding=1)
        self.dec3      = DoubleConv(base_channels*2 + base_channels*4, base_channels*2)
        self.trans_up2 = TransformerBlock(base_channels*2, emb_dim, num_heads)

        # Stage 2→1: 128 → 64, then cat with enc2(64)→128→64
        self.up2       = nn.ConvTranspose2d(base_channels*2, base_channels, 4, stride=2, padding=1)
        self.dec2      = DoubleConv(base_channels + base_channels*2, base_channels)
        self.trans_up1 = TransformerBlock(base_channels, emb_dim, num_heads)

        # Final 1×1 conv to RGB
        self.out_conv = nn.Conv2d(base_channels, in_channels, 1)

    def forward(self, x, t):
        # build time embedding once
        t_emb = self.time_mlp(t)  # [B, emb_dim]

        # —— Encode ——
        e1 = self.enc1(x)               # [B,  64, 16,16]
        e1 = self.trans1(e1, t_emb)

        x2 = self.enc2(e1)              # [B, 128,16,16]
        d2 = self.down2(x2)             # [B, 128, 8, 8]
        e2 = self.trans2(d2, t_emb)

        x3 = self.enc3(e2)              # [B, 256, 8, 8]
        d3 = self.down3(x3)             # [B, 256, 4, 4]
        e3 = self.trans3(d3, t_emb)

        x4 = self.enc4(e3)              # [B, 512, 4, 4]
        b  = self.down4(x4)             # [B, 512, 2, 2]
        b  = self.trans4(b, t_emb)

        # —— Decode ——
        u3 = self.up4(b)                           # [B, 256, 4, 4]
        u3 = torch.cat([u3, x4], dim=1)            # [B, 256+512=768, 4,4]
        u3 = self.dec4(u3)                         # [B, 256, 4, 4]
        u3 = self.trans_up3(u3, t_emb)

        u2 = self.up3(u3)                          # [B, 128, 8, 8]
        u2 = torch.cat([u2, x3], dim=1)            # [B, 128+256=384, 8,8]
        u2 = self.dec3(u2)                         # [B, 128, 8, 8]
        u2 = self.trans_up2(u2, t_emb)

        u1 = self.up2(u2)                          # [B, 64,16,16]
        u1 = torch.cat([u1, x2], dim=1)            # [B, 64+128=192,16,16]
        u1 = self.dec2(u1)                         # [B, 64,16,16]
        u1 = self.trans_up1(u1, t_emb)

        return self.out_conv(u1)                   # [B,  3,16,16]
    


## From pixart-alpha

In [4]:
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.GELU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.GELU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.GELU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(256 * 8 * 8, latent_dim)
        self.fc_logvar = nn.Linear(256 * 8 * 8, latent_dim)

    def forward(self, x):
        h = self.conv(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 256 * 8 * 8)
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.GELU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.GELU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1), nn.Tanh()
        )

    def forward(self, z):
        h = self.fc(z).view(-1, 256, 8, 8)
        return self.deconv(h)
    
def get_timestep_embedding(timesteps, dim):
    """
    Create sinusoidal embeddings.
    
    timesteps: Tensor of shape [B] (assumed integer values)
    dim: Embedding dimension
    Returns tensor of shape [B, dim]
    """
    assert len(timesteps.shape) == 1
    half_dim = dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) * -emb)
    emb = timesteps.float().unsqueeze(1) * emb.unsqueeze(0)  # [B, half_dim]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if dim % 2 == 1:  # zero pad
        emb = F.pad(emb, (0, 1))
    return emb  # shape [B, dim]

class TransformerBlock(nn.Module):
    def __init__(self, dim, n_heads):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, n_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Linear(dim * 4, dim)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.ff(self.norm2(x))
        return x
    
class UViT(nn.Module):
    def __init__(self, latent_shape=(32, 32), in_channels=4, dim=512, depth=6, heads=8):
        super().__init__()
        self.latent_shape = latent_shape  # (H, W)
        self.in_channels = in_channels
        self.dim = dim
        self.channels = in_channels  # required by the diffusion wrapper
        
        # Map each token from its channel dimension
        self.patch_embed = nn.Linear(in_channels, dim)
        self.time_embed = nn.Linear(dim, dim)  # projects the timestep embedding
        
        self.encoder = nn.ModuleList([TransformerBlock(dim, heads) for _ in range(depth)])
        self.mid_block = TransformerBlock(dim, heads)
        self.decoder = nn.ModuleList([TransformerBlock(dim, heads) for _ in range(depth)])

        self.out = nn.Linear(dim, in_channels)

    def forward(self, latents, t_emb):
        # latents: [B, C, H, W]
        B, C, H, W = latents.shape
        
        # If t_emb is a raw timestep (Long) convert it to a float embedding.
        if t_emb.dtype == torch.long or t_emb.dtype == torch.int64:
            # Create sinusoidal embeddings of size [B, dim]
            t_emb = get_timestep_embedding(t_emb, self.dim).to(latents.dtype)
        
        # Rearrange latents: treat each spatial location as a token.
        # From [B, C, H, W] → [B, H*W, C]
        x = latents.permute(0, 2, 3, 1).reshape(B, H * W, C)
        # Map channels into transformer dimension
        x = self.patch_embed(x)  # shape: [B, H*W, dim]
        
        # Add time embedding.
        # self.time_embed expects input of shape [B, dim], then output is [B, dim]
        # Unsqueeze to match token dimension: [B, 1, dim]
        x = x + self.time_embed(t_emb).unsqueeze(1)
        
        skips = []
        for block in self.encoder:
            x = block(x)
            skips.append(x)

        x = self.mid_block(x)

        for block, skip in zip(self.decoder, reversed(skips)):
            x = block(x + skip)

        x = self.out(x)  # shape: [B, H*W, C]
        x = x.view(B, H, W, C).permute(0, 3, 1, 2)  # restore shape to [B, C, H, W]
        return x

## Diffusion process

In [5]:
# normalization functions
def normalize_to_neg_one_to_one(img):
    return img * 2 - 1

def unnormalize_to_zero_to_one(t):
    return (t + 1) * 0.5

class GaussianDiffusion(nn.Module):
    def __init__(
        self,
        model: nn.Module,
        image_size: int, 
        timesteps: int,
        beta_schedule: str = 'linear',
        auto_normalize: bool = True,
        lambda_l1: float = 0.1,
    ):
        super().__init__()

        self.model = model
        self.channels = model.channels
        self.image_size = image_size
        self.num_timesteps = timesteps
        self.lambda_l1 = lambda_l1

        # 1) build beta schedule
        if beta_schedule == 'linear':
            betas = linear_beta_schedule(timesteps)
        elif beta_schedule == 'cosine':
            betas = cosine_beta_schedule(timesteps)
        else:
            raise ValueError(f'Unknown beta schedule: {beta_schedule}')
        
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

        # 2) register all buffers
        self.register_buffer('betas', betas.float())
        self.register_buffer('alphas_cumprod', alphas_cumprod.float())
        self.register_buffer('alphas_cumprod_prev', alphas_cumprod_prev.float())
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod).float())
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1 - alphas_cumprod).float())
        self.register_buffer('sqrt_recip_alphas_cumprod', torch.sqrt(1.0 / alphas_cumprod).float())
        self.register_buffer('sqrt_recipm1_alphas_cumprod', torch.sqrt(1.0 / alphas_cumprod - 1.0).float())

        # posterior q(x_{t-1} | x_t, x_0)
        posterior_variance = betas * (1 - alphas_cumprod_prev) / (1 - alphas_cumprod)
        self.register_buffer('posterior_variance', posterior_variance.float())
        self.register_buffer('posterior_log_variance_clipped', torch.log(posterior_variance.clamp(min=1e-20)).float())
        self.register_buffer('posterior_mean_coef1', (betas * torch.sqrt(alphas_cumprod_prev) / (1 - alphas_cumprod)).float())
        self.register_buffer('posterior_mean_coef2', ((1 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1 - alphas_cumprod)).float())

        # 3) Normalization functions
        self.normalize = normalize_to_neg_one_to_one if auto_normalize else (lambda x: x)
        self.unnormalize = unnormalize_to_zero_to_one if auto_normalize else (lambda x: x)

    def dynamic_threshold(self, x0, percentile=0.995):
        """ Clips x0 dynamically to avoid over/under-exposed outputs """
        s = torch.quantile(x0.abs().flatten(1), percentile, dim=1)
        s = torch.maximum(s, torch.ones_like(s))[:, None, None, None]
        return x0.clamp(-s, s) / s

    def predict_start_from_noise(self, x_t, t, noise):
        """Estimate x_0 from x_t and predicted noise"""
        return (
            extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t 
            - extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise
        )
    
    def q_posterior(self, x_start, x_t, t):
        """ Compute mean & variance of q(x_{t-1}) | x_t, x_0 """
        mean = (
            extract(self.posterior_mean_coef1, t, x_t.shape) * x_start +
            extract(self.posterior_mean_coef2, t, x_t.shape) * x_t
        )
        var = extract(self.posterior_variance, t, x_t.shape)
        log_var = extract(self.posterior_log_variance_clipped, t, x_t.shape)
        return mean, var, log_var
    
    @torch.no_grad()
    def p_sample(self, x, t: int, clip_denoised: bool = True):
        """ One diffusion reverse step """
        b = x.shape[0]
        t_batch = torch.full((b,), t, device=x.device, dtype=torch.long)
        # model predicts noise
        pred_noise = self.model(x, t_batch)
        # recover x0; optionally clamp it
        x0 = self.predict_start_from_noise(x, t_batch, pred_noise)
        x0 = self.dynamic_threshold(x0)

        # posterior mean & variance
        mean, _, log_var = self.q_posterior(x0, x, t_batch)
        noise = torch.randn_like(x) if t > 0 else 0.0
        return mean + torch.exp(0.5 * log_var) * noise

    @torch.no_grad()
    def p_sample_loop(self, shape, return_all_timesteps: bool = False):
        img = torch.randn(shape, device=self.betas.device)
        if return_all_timesteps:
            all_imgs = [img]
            for t in tqdm(reversed(range(self.num_timesteps)), desc='sampling'):
                img = self.p_sample(img, t)
                all_imgs.append(img)
            out = torch.stack(all_imgs, dim=1)
        else:
            for t in tqdm(reversed(range(self.num_timesteps)), desc='sampling'):
                img = self.p_sample(img, t)
            out = img
        return self.unnormalize(out)
    
    @torch.no_grad()
    def p_sample_ddim(self, x, t, t_prev, eta=0.0, clip_denoised=True):
        """ One DDIM sampling step from x_t to x_{t_prev}"""
        b = x.shape[0]
        t_batch = torch.full((b, ), t, device=x.device, dtype=torch.long)
        # predict noise
        pred_noise = self.model(x, t_batch)
        # pred x0
        x0 = self.predict_start_from_noise(x, t_batch, pred_noise)
        x0 = self.dynamic_threshold(x0)

        alpha_t = self.alphas_cumprod[t]
        alpha_prev = self.alphas_cumprod[t_prev] if t_prev >= 0 else torch.tensor(1.0, device=x.device)
        sqrt_alpha_t = alpha_t.sqrt()
        sqrt_alpha_prev = alpha_prev.sqrt()
        sigma_t = eta * ((1 - alpha_prev) / (1 - alpha_t) * (1 - alpha_t / alpha_prev)).sqrt()
        pred_dir = (1 - alpha_prev - sigma_t ** 2).sqrt() * pred_noise
        noise = sigma_t * torch.randn_like(x) if t_prev >= 0 else 0.0
        x_prev = sqrt_alpha_prev * x0 + pred_dir + noise

        return x_prev, x0
    
    @torch.no_grad()
    def ddim_sample_loop(self, shape, num_ddim_steps=50, eta=0.0, return_all_timesteps=False):
        """ Run the fuull DDIM sampling loop """
        device = self.betas.device
        img = torch.randn(shape, device=device)
        all_imgs = [img]

        # Create a custom timestep schedule
        ddim_timesteps = np.linspace(0, self.num_timesteps - 1, num_ddim_steps, dtype=int)

        for i in tqdm(range(num_ddim_steps - 1, -1, -1), desc='DDIM sampling'):
            t = ddim_timesteps[i]
            t_prev = ddim_timesteps[i - 1] if i > 0 else -1

            img, _ = self.p_sample_ddim(img, t, t_prev, eta=eta)
            all_imgs.append(img)
        
        out = torch.stack(all_imgs, dim=1) if return_all_timesteps else img
        return self.unnormalize(out)

    def sample(self, batch_size=16, use_ddim=False, num_ddim_steps=50, eta=0.0, return_all_timesteps=False):
        shape = (batch_size, self.channels, self.image_size, self.image_size)
        if use_ddim:
            return self.ddim_sample_loop(shape, num_ddim_steps=num_ddim_steps, eta=eta, return_all_timesteps=return_all_timesteps)
        else:
            return self.p_sample_loop(shape, return_all_timesteps=return_all_timesteps)
    
    def q_sample(self, x_start, t, noise=None):
        """ Forward noising process """
        if noise is None:
            noise = torch.randn_like(x_start)
        return (
            extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start + 
            extract(self.sqrt_one_minus_alphas_cumprod, t, x_start.shape) * noise
        )
    
    def p_losses(self, x_start, t, noise=None):
        """ MSE + L1 loss between true noise and model's prediction """
        if noise is None:
            noise = torch.randn_like(x_start)
        x_noisy = self.q_sample(x_start, t, noise)
        pred_noise = self.model(x_noisy, t)
        mse_loss = F.mse_loss(pred_noise, noise, reduction='none')
        mse_loss = mse_loss.mean(dim=list(range(1, mse_loss.ndim))) # mean over c, h, w

        # x_recon = self.predict_start_from_noise(x_noisy, t, pred_noise)
        # x_recon = x_recon.clamp(-1., -1.)
        # l1_loss = F.l1_loss(x_recon, x_start, reduction='none')
        # l1_loss = l1_loss.mean(dim=list(range(1, l1_loss.ndim)))

        # loss = mse_loss + self.lambda_l1 * l1_loss
        loss = mse_loss
        return loss.mean()
    
    def forward(self, img):
        """ Training entrypoint: sample random timesteps & return loss """
        b, c, h, w = img.shape
        assert h == self.image_size and w == self.image_size
        t = torch.randint(0, self.num_timesteps, (b, ), device=img.device)
        img = self.normalize(img)
        return self.p_losses(img, t)

## Trainer

In [ ]:
def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

def cycle(dl):
    while True:
        for data in dl:
            yield data

class Trainer:
    def __init__(
        self,
        diffusion_model: nn.Module,
        data_folder: str,
        batch_size: int = 16,
        lr: float = 1e-4,
        num_steps: int = 100000,
        grad_accum_steps: int = 1,
        ema_decay: float = 0.995,
        save_interval: int = 1000,
        num_samples: int = 25,
        results_folder: str = './results',
        use_ddim = False,
        num_ddim_steps=50,
        eta = 0.0
    ):
        # Accelerator
        self.accelerator = Accelerator(mixed_precision='fp16')
        self.device = self.accelerator.device

        # Training State
        self.batch_size       = batch_size
        self.grad_accum_steps = grad_accum_steps
        self.num_steps        = num_steps
        self.save_interval    = save_interval
        self.num_samples      = num_samples

        # model, optimizer, EMA
        self.model = diffusion_model.to(self.device)
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=self.num_steps, eta_min=1e-6)
        self.model, self.optimizer = self.accelerator.prepare(
            self.model, self.optimizer
        )
        
        # Use EMA on the raw model
        self.ema = EMA(self.accelerator.unwrap_model(self.model), beta=ema_decay)

        # Data
        ds = CustomLatentDataset(data_folder, diffusion_model.image_size, vae)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
        self.dl = cycle(self.accelerator.prepare(dl))

        # checkpoints & samples
        self.results_folder = Path(results_folder)
        self.results_folder.mkdir(parents=True, exist_ok=True)
        self.step = 0
        self.use_ddim = use_ddim
        self.num_ddim_steps = num_ddim_steps
        self.eta = eta
    
    def save(self, milestone: int):
        """Save model, optimizer, EMA and step counter."""
        if not self.accelerator.is_main_process:
            return
        ckpt = self.results_folder / f'model-{milestone}.pt'
        data = {
            'step': self.step,
            'model': self.accelerator.get_state_dict(self.model),
            'optimizer': self.optimizer.state_dict(),
            'ema':       self.ema.state_dict(),
        }
        torch.save(data, ckpt)

    def load(self, ckpt_path: str):
        """Load all state (model, optimizer, EMA, step)."""
        data = torch.load(ckpt_path, map_location=self.device)
        raw_model = self.accelerator.unwrap_model(self.model)
        raw_model.load_state_dict(data['model'])
        self.optimizer.load_state_dict(data['optimizer'])
        self.ema.load_state_dict(data['ema'])
        self.step = data['step']

    def _sample_and_save(self, milestone: int):
        """Generate `num_samples` via EMA model and save grid."""
        self.ema.ema_model.eval()
        batches = num_to_groups(self.num_samples, self.batch_size)
        imgs = torch.cat([
            self.ema.ema_model.sample(batch_size=n, use_ddim=self.use_ddim, num_ddim_steps=self.num_ddim_steps, eta=self.eta) for n in batches
        ], dim=0)
        path = self.results_folder / f'sample-{milestone}.png'
        vutils.save_image(imgs, path, nrow=int(math.sqrt(self.num_samples)))

    def train(self):
        """Run the training loop with gradient accumulation, EMA updates, and periodic sampling."""
        pbar = tqdm(total=self.num_steps, initial=self.step, disable=True)
        while self.step < self.num_steps:
            total_loss = 0.0

            # gradient accumulation
            for _ in range(self.grad_accum_steps):
                batch = next(self.dl).to(self.device)
                with self.accelerator.autocast():
                    loss = self.model(batch) / self.grad_accum_steps
                total_loss += loss.item()
                self.accelerator.backward(loss)

            # optimizer step
            self.accelerator.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()
            self.step += 1

            # EMA & sampling
            if self.accelerator.is_main_process:
                self.ema.update()
                if self.step % self.save_interval == 0:
                    milestone = self.step // self.save_interval
                    self._sample_and_save(milestone)
                    self.save(milestone)
            if self.step % 10 == 0 and self.accelerator.is_main_process:
                print(f"[Step {self.step}] loss: {total_loss:.4f}")

            pbar.set_description(f'loss: {total_loss:.4f}')
            pbar.update(1)

        if self.accelerator.is_main_process:
            print('Training complete.')

    def inference(self, total: int = 1000, output_path: str = './submission'):
        """
        Generate `total` images using the same batch size as training
        (self.batch_size, e.g. 16), and save them to disk.
        """
        Path(output_path).mkdir(parents=True, exist_ok=True)
        count = 0
        batch_size = self.batch_size  # manually fixed at 16

        with torch.no_grad():
            while count < total:
                n = min(batch_size, total - count)

                # clear any leftover GPU memory
                if torch.cuda.is_available():
                    gc.collect()
                    torch.cuda.empty_cache()

                # sample n images
                imgs = self.ema.ema_model.sample(batch_size=n, use_ddim=self.use_ddim, num_ddim_steps=self.num_ddim_steps, eta=self.eta)

                # move to CPU and save
                imgs = imgs.cpu()
                for img in imgs:
                    count += 1
                    vutils.save_image(img, f"{output_path}/{count}.jpg")

                # clean up 
                del imgs
                if torch.cuda.is_available():
                    gc.collect()
                    torch.cuda.empty_cache()

        print("Inference complete.")

## Usage

In [8]:
gc.collect()
torch.cuda.empty_cache()

In [9]:
# model = TransformerBackbone(
#     in_channels=3, img_size=16, patch_size=1,
#     emb_dim=1024, depth=8, num_heads=8, time_emb_dim=128
# )

# model = UNetTransformer(
#     in_channels=3,
#     base_channels=64,
#     emb_dim=256,
#     num_heads=4,
#     time_emb_dim=128,
# )

model = UViT((16, 16), 3, dim=256, depth=6, heads=8)

diffusion = GaussianDiffusion(
    model=model, image_size=16, timesteps=1000,
    beta_schedule='cosine', auto_normalize=True
)
trainer = Trainer(
    diffusion_model=diffusion,
    data_folder = './sprites.npy',
    batch_size=32, lr=1e-4, num_steps=50000,
    grad_accum_steps=1, ema_decay=0.999,
    save_interval=1000, num_samples=25,
    results_folder='./results',
    use_ddim=False, num_ddim_steps=50, eta=0.0
)


In [10]:
total_params = sum(p.numel() for p in model.parameters())
print("Params: ", total_params/1e6, "M")

Params:  10.334467 M


In [11]:
trainer.train()

[Step 10] loss: 5.7415
[Step 20] loss: 2.0250
[Step 30] loss: 1.0543
[Step 40] loss: 0.8597
[Step 50] loss: 0.6562
[Step 60] loss: 0.5841
[Step 70] loss: 0.4760
[Step 80] loss: 0.5589
[Step 90] loss: 0.4919
[Step 100] loss: 0.5668
[Step 110] loss: 0.4235
[Step 120] loss: 0.4329
[Step 130] loss: 0.4620
[Step 140] loss: 0.4182
[Step 150] loss: 0.5452
[Step 160] loss: 0.3483
[Step 170] loss: 0.4825
[Step 180] loss: 0.5201
[Step 190] loss: 0.3940
[Step 200] loss: 0.4752
[Step 210] loss: 0.4016
[Step 220] loss: 0.4636
[Step 230] loss: 0.3973
[Step 240] loss: 0.3456
[Step 250] loss: 0.3587
[Step 260] loss: 0.4227
[Step 270] loss: 0.3488
[Step 280] loss: 0.4658
[Step 290] loss: 0.4055
[Step 300] loss: 0.3921
[Step 310] loss: 0.4518
[Step 320] loss: 0.3281
[Step 330] loss: 0.3635
[Step 340] loss: 0.3296
[Step 350] loss: 0.4560
[Step 360] loss: 0.4826
[Step 370] loss: 0.4448
[Step 380] loss: 0.3343
[Step 390] loss: 0.2936
[Step 400] loss: 0.4284
[Step 410] loss: 0.2878
[Step 420] loss: 0.2962
[

sampling: 0it [00:00, ?it/s]

[Step 1000] loss: 0.2902
[Step 1010] loss: 0.2847
[Step 1020] loss: 0.3017
[Step 1030] loss: 0.2664
[Step 1040] loss: 0.2650
[Step 1050] loss: 0.2778
[Step 1060] loss: 0.2289
[Step 1070] loss: 0.2946
[Step 1080] loss: 0.2845
[Step 1090] loss: 0.2071
[Step 1100] loss: 0.2505
[Step 1110] loss: 0.2999
[Step 1120] loss: 0.3292
[Step 1130] loss: 0.2806
[Step 1140] loss: 0.3170
[Step 1150] loss: 0.2414
[Step 1160] loss: 0.2475
[Step 1170] loss: 0.3011
[Step 1180] loss: 0.2982
[Step 1190] loss: 0.2678
[Step 1200] loss: 0.2122
[Step 1210] loss: 0.2548
[Step 1220] loss: 0.2431
[Step 1230] loss: 0.2515
[Step 1240] loss: 0.2530
[Step 1250] loss: 0.2582
[Step 1260] loss: 0.2410
[Step 1270] loss: 0.2247
[Step 1280] loss: 0.2010
[Step 1290] loss: 0.2267
[Step 1300] loss: 0.2834
[Step 1310] loss: 0.2206
[Step 1320] loss: 0.2566
[Step 1330] loss: 0.2398
[Step 1340] loss: 0.2713
[Step 1350] loss: 0.2447
[Step 1360] loss: 0.2195
[Step 1370] loss: 0.2673
[Step 1380] loss: 0.2251
[Step 1390] loss: 0.2077


sampling: 0it [00:00, ?it/s]

[Step 2000] loss: 0.2010
[Step 2010] loss: 0.2711
[Step 2020] loss: 0.2977
[Step 2030] loss: 0.2233
[Step 2040] loss: 0.1921
[Step 2050] loss: 0.2432
[Step 2060] loss: 0.2433
[Step 2070] loss: 0.1935
[Step 2080] loss: 0.2157
[Step 2090] loss: 0.2113
[Step 2100] loss: 0.2520
[Step 2110] loss: 0.2082
[Step 2120] loss: 0.2330
[Step 2130] loss: 0.1976
[Step 2140] loss: 0.2190
[Step 2150] loss: 0.2361
[Step 2160] loss: 0.2676
[Step 2170] loss: 0.2452
[Step 2180] loss: 0.2267
[Step 2190] loss: 0.2390
[Step 2200] loss: 0.2157
[Step 2210] loss: 0.1870
[Step 2220] loss: 0.1963
[Step 2230] loss: 0.2199
[Step 2240] loss: 0.2371
[Step 2250] loss: 0.2463
[Step 2260] loss: 0.2572
[Step 2270] loss: 0.2103
[Step 2280] loss: 0.2244
[Step 2290] loss: 0.2763
[Step 2300] loss: 0.2367
[Step 2310] loss: 0.2051
[Step 2320] loss: 0.2498
[Step 2330] loss: 0.2073
[Step 2340] loss: 0.2441
[Step 2350] loss: 0.2211
[Step 2360] loss: 0.1999
[Step 2370] loss: 0.2429
[Step 2380] loss: 0.2169
[Step 2390] loss: 0.2669


...

[Step 48000] loss: 0.2017
[Step 48010] loss: 0.1958
[Step 48020] loss: 0.1930
[Step 48030] loss: 0.2051
[Step 48040] loss: 0.2195
[Step 48050] loss: 0.1925
[Step 48060] loss: 0.1864
[Step 48070] loss: 0.1387
[Step 48080] loss: 0.2068
[Step 48090] loss: 0.2087
[Step 48100] loss: 0.1909
[Step 48110] loss: 0.1902
[Step 48120] loss: 0.1832
[Step 48130] loss: 0.2198
[Step 48140] loss: 0.1798
[Step 48150] loss: 0.1786
[Step 48160] loss: 0.1920
[Step 48170] loss: 0.1927
[Step 48180] loss: 0.2591
[Step 48190] loss: 0.1942
[Step 48200] loss: 0.2100
[Step 48210] loss: 0.1863
[Step 48220] loss: 0.1858
[Step 48230] loss: 0.1904
[Step 48240] loss: 0.1801
[Step 48250] loss: 0.2079
[Step 48260] loss: 0.1664
[Step 48270] loss: 0.1749
[Step 48280] loss: 0.1840
[Step 48290] loss: 0.1997
[Step 48300] loss: 0.1996
[Step 48310] loss: 0.1815
[Step 48320] loss: 0.1813
[Step 48330] loss: 0.2380
[Step 48340] loss: 0.2313
[Step 48350] loss: 0.2043
[Step 48360] loss: 0.2052
[Step 48370] loss: 0.2202
[Step 48380]

sampling: 0it [00:00, ?it/s]

[Step 49000] loss: 0.1670
[Step 49010] loss: 0.2011
[Step 49020] loss: 0.1817
[Step 49030] loss: 0.1899
[Step 49040] loss: 0.2210
[Step 49050] loss: 0.1970
[Step 49060] loss: 0.1814
[Step 49070] loss: 0.1966
[Step 49080] loss: 0.1904
[Step 49090] loss: 0.2125
[Step 49100] loss: 0.2252
[Step 49110] loss: 0.2013
[Step 49120] loss: 0.2187
[Step 49130] loss: 0.1993
[Step 49140] loss: 0.2195
[Step 49150] loss: 0.1997
[Step 49160] loss: 0.2421
[Step 49170] loss: 0.2029
[Step 49180] loss: 0.2066
[Step 49190] loss: 0.2030
[Step 49200] loss: 0.1984
[Step 49210] loss: 0.2371
[Step 49220] loss: 0.1701
[Step 49230] loss: 0.1872
[Step 49240] loss: 0.1996
[Step 49250] loss: 0.1800
[Step 49260] loss: 0.2194
[Step 49270] loss: 0.1780
[Step 49280] loss: 0.2175
[Step 49290] loss: 0.1974
[Step 49300] loss: 0.2351
[Step 49310] loss: 0.2132
[Step 49320] loss: 0.2008
[Step 49330] loss: 0.2148
[Step 49340] loss: 0.2131
[Step 49350] loss: 0.1927
[Step 49360] loss: 0.1916
[Step 49370] loss: 0.2050
[Step 49380]

sampling: 0it [00:00, ?it/s]

[Step 50000] loss: 0.2231
Training complete.


## Inference

In [ ]:
ckpt = './UNetTransformer-emb256.pt'
trainer.load(ckpt)
# trainer.inference()

In [ ]:
diffusion = trainer.model
with torch.no_grad():
    all_steps = diffusion.sample(batch_size=1, return_all_timesteps=True)
# shape [1, T+1, C, H, W]

imgs = all_steps.squeeze(0).permute(0, 2, 3, 1).cpu().numpy()
imgs = unnormalize_to_zero_to_one(imgs)

timesteps = [0, diffusion.num_timesteps // 4, diffusion.num_timesteps // 2,
             3 * diffusion.num_timesteps // 4, diffusion.num_timesteps]
sel = [imgs[t] for t in timesteps]

# Plot
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for ax, im, step in zip(axes, sel, timesteps):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(f"Step {step}")
plt.tight_layout()
plt.show()